In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
from imports import *
from functions import *

Connection to PostgreSQL

In [3]:
path_postgresql_creds = r"C:\Users\f.gionnane\Documents\Data Engineering\Credentials\postgresql_creds.json"

with open(path_postgresql_creds, 'r') as file:
    content = json.load(file)
    user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]

db = "Oceanography_ML_Project"
schema_bronze = "Bronze"
schema_silver = "Silver"

# Créer l'engine PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")
conn = engine.connect()

Charger les Données des Tables de la couche de Bronze

In [4]:
# Charger les métadonnées du schéma existant
metadata = MetaData(schema=schema_bronze)

print("\n🔍 Chargement des métadonnées du schéma...")
metadata.reflect(bind=conn)
print("✅ Métadonnées chargées avec succès.\n")

# Récupérer les noms des tables
table_names = [t.name for t in metadata.sorted_tables]
print(f"🔢 Nombre total de tables dans le schéma : {len(table_names)}\n")

# Filtrer les tables en fonction du contenu de leur nom
marine_tables = {t for t in table_names if "marine" in t.lower()}
meteo_tables = {t for t in table_names if "meteo" in t.lower()}
buoys_data_table = {t for t in table_names if "buoy" in t.lower()}

print(f"🌊 Tables marines trouvées : {len(marine_tables)}")
print(f"🌧️ Tables météo trouvées : {len(meteo_tables)}")
print(f"🐋 Tables de bouées trouvées : {len(buoys_data_table)}\n")

# Initialiser le dictionnaire des résultats
buoys_datas = {}

# Compteurs pour suivre le nombre de tables chargées avec succès
marine_data_count = 0
meteo_data_count = 0
buoys_data_count = 0

# Compteur pour le nombre total de lignes
total_marine_rows = 0
total_meteo_rows = 0
total_buoys_rows = 0  # Changer ici pour compter le nombre de lignes (bouées)

# Vérifier et récupérer les données de la table "buoys_datas"
if buoys_data_table:
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print("🔄 Chargement des données de la table 'buoys_datas'...")

    try:
        buoys_datas_raw = fetch_table_data(schema=schema_bronze, conn=conn, table_name=next(iter(buoys_data_table)), as_df=True)

        if buoys_datas_raw is not None:
            print("📦 Données récupérées pour 'buoys_datas'.")

            # Conversion JSON → dict si nécessaire
            if isinstance(buoys_datas_raw, str):
                buoys_datas_raw = json.loads(buoys_datas_raw)

            elif isinstance(buoys_datas_raw, pd.DataFrame) and "Station ID" in buoys_datas_raw.columns:
                # Convertir en dictionnaire avec "Station ID" comme clé
                buoys_datas_raw = buoys_datas_raw.set_index("Station ID").to_dict(orient="index")

            # Ajouter au dictionnaire principal directement avec les Station ID comme clés
            buoys_datas.update(buoys_datas_raw)
            buoys_data_count += 1
            total_buoys_rows += len(buoys_datas_raw)  # Compter le nombre de bouées
            print(f"✅ Table 'buoys_datas' chargée avec succès! Nombre de bouées (lignes) : {total_buoys_rows}\n")
        else:
            print("⚠️ Aucun résultat trouvé dans 'buoys_datas'.\n")

    except Exception as e:
        print(f"❌ Erreur lors du chargement de 'buoys_datas': {e}\n")

# Associer les tables marine et meteo en fonction du station_id et récupérer leurs données
for table_set, label, icon, counter, total_rows in [
    (marine_tables, "Marine", "🌊", marine_data_count, total_marine_rows),
    (meteo_tables, "Meteo", "🌧️", meteo_data_count, total_meteo_rows)
]:
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    for table_name in table_set:
        print(f"🔄 Chargement des données pour la table {label} : {table_name}...")

        try:
            station_id = table_name.split("_")[1]

            # Vérifier si la station existe déjà dans buoys_datas, sinon initialiser un dictionnaire
            if station_id not in buoys_datas:
                buoys_datas[station_id] = {}

            # Récupérer les données
            data = fetch_table_data(schema=schema_bronze, conn=conn, table_name=table_name, as_df=True)

            if data is not None:
                print(f"📦 Données récupérées pour la station {station_id} ({label}).")

                if isinstance(data, str):
                    data = pd.DataFrame(json.loads(data))
                elif isinstance(data, dict):
                    data = pd.DataFrame(data)

                # Ajouter les données au dictionnaire de bouées sous la station_id
                buoys_datas[station_id][f"{label} DataFrame"] = data
                counter += 1
                total_rows += len(data)  # Ajouter le nombre de lignes collectées
                print(f"{icon} Données {label} chargées pour la station {station_id}! Nombre de lignes collectées : {len(data)}\n")
            else:
                print(f"⚠️ Aucun résultat trouvé pour la station {station_id} ({label}).\n")

        except Exception as e:
            print(f"❌ Erreur lors du chargement des données {label} pour {table_name} : {e}\n")

    # Mise à jour des compteurs après le chargement des données pour chaque catégorie
    if label == "Marine":
        marine_data_count = counter
        total_marine_rows = total_rows
    elif label == "Meteo":
        meteo_data_count = counter
        total_meteo_rows = total_rows

# Finalement, afficher un récapitulatif global
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"🏆 Chargement des données terminé avec succès !")
print(f"🐋 Total des données bouées chargées : {buoys_data_count} - Nombre de bouées (lignes) : {total_buoys_rows}")
print(f"🌊 Total des données marines chargées : {marine_data_count} - Nombre total de lignes : {total_marine_rows}")
print(f"🌧️ Total des données météorologiques chargées : {meteo_data_count} - Nombre total de lignes : {total_meteo_rows}")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")


🔍 Chargement des métadonnées du schéma...
✅ Métadonnées chargées avec succès.

🔢 Nombre total de tables dans le schéma : 79

🌊 Tables marines trouvées : 39
🌧️ Tables météo trouvées : 39
🐋 Tables de bouées trouvées : 1

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔄 Chargement des données de la table 'buoys_datas'...
📦 Données récupérées pour 'buoys_datas'.
✅ Table 'buoys_datas' chargée avec succès! Nombre de bouées (lignes) : 39

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔄 Chargement des données pour la table Marine : station_51002_marine_southwest hawaii...
📦 Données récupérées pour la station 51002 (Marine).
🌊 Données Marine chargées pour la station 51002! Nombre de lignes collectées : 6481

🔄 Chargement des données pour la table Marine : station_MDRM1_marine_mt_ desert rock, me...
📦 Données récupérées pour la station MDRM1 (Marine).
🌊 Données Marine chargées pour la station MDRM1! Nombre de lignes collectées : 1091

🔄 Chargement des données pour la table Marine : station_LONF1_marine_long key

Fuse all the Dataframe in one Final Dataframe

In [ ]:
list_silver_merged_df = []  # List to store merged DataFrames
list_failed_dfs = []        # List to store failed DataFrame pairs

number_marine_data = 0
number_meteo_data = 0
number_merged_data = 0

marine_data_conversion = 0
meteo_data_conversion = 0

# Loop through each station in the dictionary
for idx, (station_id, tables) in enumerate(buoys_datas.items()):
    print(f"\n🔄 Processing station {station_id} ({idx + 1}/{len(buoys_datas)})...")

    # Check if the station's buoy data is excluded from normal processing
    # Retrieve weather and marine DataFrames for the station
    df_meteo = buoys_datas[station_id].get("Meteo DataFrame", None)
    df_marine = buoys_datas[station_id].get("Marine DataFrame", None)
    
    # Ensure all necessary data is present
    if df_meteo is None or df_marine is None or buoys_datas[station_id] is None:
        print(f"⚠️ Missing data for station {station_id}. Skipping this station.")
        list_failed_dfs.append(station_id)
        continue

    print(f"📦 Data retrieved for station {station_id}:\n")
    number_marine_data += int(df_marine.shape[0])
    number_meteo_data += int(df_meteo.shape[0])

    # Add buoy coordinates (Lat/Lon) to marine data
    print("📍 Merging buoy coordinates (Lat/Lon) with marine data...")

    #### ADDING COORDINATES TO df_marine
    df_marine["Lat"] = buoys_datas[station_id]["Lat"]
    df_marine["Lon"] = buoys_datas[station_id]["Lon"]

    print(f"🌐 Coordinates (Lat/Lon) added for station {station_id}: {df_marine['Lat'].iloc[0]} / {df_marine['Lon'].iloc[0]}.")
    

                ### HANDLING DATA NULL VALUE ###


    ### HANDLING MARINE DATA NULL VALUE
    print(f"Handling Null Values in Marine data for Buoy {station_id}")
    try:
        df_marine = handle_null_values(df_marine)
    except Exception as e:
        print(f"Error: {e}")
    
    ### HANDLING METEO DATA NULL VALUE

    print(f"Handling Null Values in weather data for Buoy {station_id}")
    try:
        df_meteo = handle_null_values(df_meteo)
    except Exception as e:
        print(f"Error: {e}")


                    ### PROCESS AND RESAMPLE ###

    ### PROCESS AND RESAMPLE MARINE DATA
    print(f"🔁 Processing and resampling marine data for station {station_id}...")
    df_marine = process_and_resample(df_marine, column_name='time')
    print(f"✅ Marine data resampled for station {station_id}.")

    ### PROCESS AND RESAMPLE METEO DATA
    print(f"🔁 Processing and resampling weather data for station {station_id}...")
    df_meteo = process_and_resample(df_meteo, column_name='date')
    print(f"✅ Weather data resampled for station {station_id}.")


                    #### AUTOCONVERT PROCESS ###

    ### Trying Autoconvert on Marine Data
    print(f"🔁 Processing to Marine Data Conversion for station {station_id}...")
    try:
        df_marine = auto_convert(df_marine)
        print(f"✅ Marine Data Successfully Converted for Station {station_id}.")
        marine_data_conversion += 0

    except Exception as e:
        print(f"Conversion Failed :\nError: {e}")

    ### Trying Autoconvert on Meteo Data
    print(f"🔁 Processing to Meteo Data Conversion for station {station_id}...")
    try:
        df_meteo = auto_convert(df_meteo)
        print(f"✅ Meteo Data Successfully Converted for Station {station_id}.")
        meteo_data_conversion += 1

    except Exception as e:
        print(f"Conversion Failed :\nError: {e}")

    ##### MERGING DATAFRAMES

    # Merge marine and weather DataFrames
    print(f"🔗 Merging marine and weather data for station {station_id}...")
    df_merged = pd.merge(df_marine, df_meteo, on='Datetime', how='inner')
    buoys_datas[str(station_id)]["Merged Dataframe"] = df_merged
    df_merged_row_count = int(df_merged.shape[0])
    number_merged_data += df_merged_row_count
    
    print(f"🔢 Buoy {station_id} Merged Dataframe row count: {df_merged_row_count}"
            
          f"💾 Merged data added to dictionary for station {station_id}.")


    list_silver_merged_df.append(df_merged)

# Final merge of all DataFrames
print("🔀 Merging all DataFrames into a final DataFrame...")
# Display the size of DataFrames before final merge

# Extract only the merged DataFrames from each station
dataframes_to_concat = [station_data["Merged Dataframe"] for station_data in buoys_datas.values()]

# Merge the DataFrames
df_final = pd.concat(dataframes_to_concat, ignore_index=True)

# Print some details about the final merge
print(f"📝 Final merged DataFrame size: {df_final.shape}")
print("✅ Final merge completed successfully!")

# Final summary
print("\n⭐🏆 Processing complete!")
print(f"🔢 Total stations processed: {len(buoys_datas)}")
print(f"Marine data rows collected = {number_marine_data}\nMeteo data rows collected = {number_meteo_data}")
print(f"Marine Data Successfully Converted:  {marine_data_conversion}")
print(f"Meteo Data Successfully Converted:  {meteo_data_conversion}")

if df_final is not None and not df_final.empty:
    print(f"Total Number of merged rows: {number_merged_data}")
    print(f"Final DataFrame rows number: {df_final.shape[0]}")

else:
    print("The DataFrame is either None or empty.")

print(f"❌ Number of failed stations: {len(list_failed_dfs)}")
if list_failed_dfs:
    print(f"⚠️ Failed stations: {', '.join(list_failed_dfs)}")


🔄 Processing station 41008 (1/39)...
📦 Data retrieved for station 41008:

📍 Merging buoy coordinates (Lat/Lon) with marine data...
🌐 Coordinates (Lat/Lon) added for station 41008: 31.40N / 80.87W.
Handling Null Values in Marine data for Buoy 41008
Dropped columns (100% missing): visibility, water_level_above_mean
Dropped columns (>50% missing): dominant_wave_period, 3hr_pressure_tendency
Imputed columns (<50% missing, median): average_wave_period, dominant_wave_direction, pressure, air_temperature, water_temperature, dewpoint
🔁 Processing and resampling marine data for station 41008...
✅ Marine data resampled for station 41008.
Handling Null Values in weather data for Buoy 41008
Skipped non-numeric columns: temperature_2m, relative_humidity_2m, dew_point_2m, precipitation, rain, showers, pressure_msl, surface_pressure, cloud_cover, cloud_cover_low, cloud_cover_mid, cloud_cover_high, visibility, wind_speed_10m, soil_temperature_0cm, soil_moisture_0_to_1cm
🔁 Processing and resampling we

In [34]:
df_final.dtypes

id_x                                           int64
wind_direction                               float64
wind_speed                                   float64
wind_gust                                    float64
wave_height                                  float64
average_wave_period                          float64
dominant_wave_direction                      float64
pressure                                     float64
air_temperature                              float64
water_temperature                            float64
dewpoint                                     float64
Datetime                   datetime64[ns, UTC+01:00]
Station ID                                    object
Lat                                           object
Lon                                           object
id_y                                           int64
temperature_2m                                object
relative_humidity_2m                          object
dew_point_2m                                  

In [33]:
display_null_counts(df_final)

id_x                     0   / 40857
wind_direction           0   / 40857
wind_speed               0   / 40857
wind_gust                1091/ 40857
wave_height              13091/ 40857
average_wave_period      13091/ 40857
dominant_wave_direction  15275/ 40857
pressure                 0   / 40857
air_temperature          5460/ 40857
water_temperature        10900/ 40857
dewpoint                 9822/ 40857
Datetime                 0   / 40857
Station ID               0   / 40857
Lat                      0   / 40857
Lon                      0   / 40857
id_y                     0   / 40857
temperature_2m           0   / 40857
relative_humidity_2m     0   / 40857
dew_point_2m             0   / 40857
precipitation            0   / 40857
rain                     0   / 40857
showers                  0   / 40857
pressure_msl             0   / 40857
surface_pressure         0   / 40857
cloud_cover              0   / 40857
cloud_cover_low          0   / 40857
cloud_cover_mid          0   / 408

In [23]:
df_final.head(10)

,id_x,wind_direction,wind_speed,wind_gust,wave_height,average_wave_period,dominant_wave_direction,pressure,air_temperature,water_temperature,...,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day,3hr_pressure_tendency
0,3,240.0,6.0,7.0,0.8,4.8,104.0,1020.3,14.2,15.3,...,0.0,0.0,0.0,0.0,34300.0,22.06999969482422,6.739999771118164,0.2800000011920929,0.0,NaN
1,9,230.0,7.0,8.0,0.8,4.8,104.0,1019.9,14.2,15.3,...,56.0,0.0,56.0,0.0,38000.0,25.809999465942383,7.039999961853027,0.2800000011920929,0.0,NaN
2,15,220.0,7.0,9.0,0.8,4.8,104.0,1019.6,14.4,15.3,...,0.0,0.0,0.0,0.0,41200.0,27.290000915527344,7.539999961853027,0.2800000011920929,0.0,NaN
3,21,220.0,6.0,8.0,0.8,4.8,104.0,1019.7,15.0,15.3,...,99.0,0.0,99.0,0.0,40500.0,21.260000228881836,7.889999866485596,0.2800000011920929,0.0,NaN
4,27,210.0,6.0,7.0,0.8,4.8,104.0,1019.7,14.8,15.4,...,95.0,69.0,95.0,0.0,43300.0,24.739999771118164,7.889999866485596,0.2800000011920929,0.0,NaN
5,33,210.0,5.0,6.0,0.8,4.8,104.0,1020.2,14.6,15.4,...,79.0,1.0,79.0,0.0,41000.0,28.350000381469727,8.140000343322754,0.2800000011920929,0.0,NaN
6,39,220.0,5.0,6.0,0.8,4.8,104.0,1020.8,14.4,15.5,...,99.0,0.0,99.0,91.0,38300.0,28.350000381469727,7.539999961853027,0.2800000011920929,0.0,NaN
7,45,210.0,5.0,6.0,0.8,4.8,104.0,1020.7,14.1,15.5,...,55.0,0.0,55.0,1.0,38700.0,25.170000076293945,7.690000057220459,0.2800000011920929,0.0,NaN
8,51,190.0,6.0,7.0,0.8,4.8,104.0,1020.6,13.9,15.5,...,60.0,0.0,60.0,24.0,41500.0,27.5,7.789999961853027,0.2800000011920929,0.0,NaN
9,2080,200.0,5.0,6.0,0.8,4.8,104.0,1016.2,14.5,13.7,...,99.0,99.0,98.0,0.0,12200.0,7.570000171661377,2.8399999141693115,0.33000001311302185,1.0,NaN


In [24]:
df_final.columns

Index(['id_x', 'wind_direction', 'wind_speed', 'wind_gust', 'wave_height',
       'average_wave_period', 'dominant_wave_direction', 'pressure',
       'air_temperature', 'water_temperature', 'dewpoint', 'Datetime',
       'Station ID', 'Lat', 'Lon', 'id_y', 'temperature_2m',
       'relative_humidity_2m', 'dew_point_2m', 'precipitation', 'rain',
       'showers', 'pressure_msl', 'surface_pressure', 'cloud_cover',
       'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high', 'visibility',
       'wind_speed_10m', 'soil_temperature_0cm', 'soil_moisture_0_to_1cm',
       'is_day', '3hr_pressure_tendency'],
      dtype='object')

In [26]:
col_to_rename={'temperature_2m': 'T°(C°)',  'relative_humidity_2m': 'Relative Humidity (%)',
 'dew_point_2m': 'Dew Point (°C)', 'precipitation': 'Precipitation (mm)',  'pressure_msl':' Sea Level Pressure (hPa)', 
 'cloud_cover_low':'Low Clouds (%)', 'cloud_cover_mid' : 'Middle Clouds (%)',	 'cloud_cover_high' : 'High Clouds (%)', 
 'visibility' : ' Visibility (%)',  'wind_speed_10m' : 'Wind Speed (km/h)',  'wind_direction': 'Wind Direction (°)',
 'wind_speed': 'Wind Speed (km/h)','wind_gust': 'Wind Gusts (km/h)', 'wave_height': 'Wave Height (m)',  'average_wave_period': 'Average Wave Period (s)',
 'dominant_wave_direction': 'Dominant Wave Direction (°)','pressure': 'Pressure (hPA)',
 'air_temperature': 'Air T°','water_temperature': 'Water T°'}
df_final = rename_columns(df_final, col_to_rename)
df_final.columns

⚠️ No columns to rename from this spec: {'temperature_2m': 'T°(C°)', 'relative_humidity_2m': 'Relative Humidity (%)', 'dew_point_2m': 'Dew Point (°C)', 'precipitation': 'Precipitation (mm)', 'pressure_msl': ' Sea Level Pressure (hPa)', 'cloud_cover_low': 'Low Clouds (%)', 'cloud_cover_mid': 'Middle Clouds (%)', 'cloud_cover_high': 'High Clouds (%)', 'visibility': ' Visibility (%)', 'wind_speed_10m': 'Wind Speed (km/h)', 'wind_direction': 'Wind Direction (°)', 'wind_speed': 'Wind Speed (km/h)', 'wind_gust': 'Wind Gusts (km/h)', 'wave_height': 'Wave Height (m)', 'average_wave_period': 'Average Wave Period (s)', 'dominant_wave_direction': 'Dominant Wave Direction (°)', 'pressure': 'Pressure (hPA)', 'air_temperature': 'Air T°', 'water_temperature': 'Water T°'}


Index(['id_x', 'Wind Direction (°)', 'Wind Speed (km/h)', 'Wind Gusts (km/h)',
       'Wave Height (m)', 'Average Wave Period (s)',
       'Dominant Wave Direction (°)', 'Pressure (hPA)', 'Air T°', 'Water T°',
       'dewpoint', 'Datetime', 'Station ID', 'Lat', 'Lon', 'id_y', 'T°(C°)',
       'Relative Humidity (%)', 'Dew Point (°C)', 'Precipitation (mm)', 'rain',
       'showers', ' Sea Level Pressure (hPa)', 'surface_pressure',
       'cloud_cover', 'Low Clouds (%)', 'Middle Clouds (%)', 'High Clouds (%)',
       ' Visibility (%)', 'Wind Speed (km/h)', 'soil_temperature_0cm',
       'soil_moisture_0_to_1cm', 'is_day', '3hr_pressure_tendency'],
      dtype='object')

In [27]:
display_null_counts(df_final)

id_x                     0   / 40857
Wind Direction (°)       0   / 40857
Wind Speed (km/h)        0   / 40857
Wind Gusts (km/h)        1091/ 40857
Wave Height (m)          13091/ 40857
Average Wave Period (s)  13091/ 40857
Dominant Wave Direction (°)15275/ 40857
Pressure (hPA)           0   / 40857
Air T°                   5460/ 40857
Water T°                 10900/ 40857
dewpoint                 9822/ 40857
Datetime                 0   / 40857
Station ID               0   / 40857
Lat                      0   / 40857
Lon                      0   / 40857
id_y                     0   / 40857
T°(C°)                   0   / 40857
Relative Humidity (%)    0   / 40857
Dew Point (°C)           0   / 40857
Precipitation (mm)       0   / 40857
rain                     0   / 40857
showers                  0   / 40857
 Sea Level Pressure (hPa)0   / 40857
surface_pressure         0   / 40857
cloud_cover              0   / 40857
Low Clouds (%)           0   / 40857
Middle Clouds (%)        0   / 4

In [31]:
df_final_col_sample = df_final[["Wind Gusts (km/h)", "Wave Height (m)", "Average Wave Period (s)", "Dominant Wave Direction (°)","Pressure (hPA)", "Air T°", "Water T°", "dewpoint"]]

print(df_final_col_sample.dtypes)
df_final_col_sample.head()

Wind Gusts (km/h)              float64
Wave Height (m)                float64
Average Wave Period (s)        float64
Dominant Wave Direction (°)    float64
Pressure (hPA)                 float64
Air T°                         float64
Water T°                       float64
dewpoint                       float64
dtype: object


,Wind Gusts (km/h),Wave Height (m),Average Wave Period (s),Dominant Wave Direction (°),Pressure (hPA),Air T°,Water T°,dewpoint
0,7.0,0.8,4.8,104.0,1020.3,14.2,15.3,10.9
1,8.0,0.8,4.8,104.0,1019.9,14.2,15.3,10.9
2,9.0,0.8,4.8,104.0,1019.6,14.4,15.3,10.9
3,8.0,0.8,4.8,104.0,1019.7,15.0,15.3,11.5
4,7.0,0.8,4.8,104.0,1019.7,14.8,15.4,11.4


In [11]:
print(f"Colonnes df_marine:\n{df_marine.columns}\n\nColonnes df_meteo:\n{df_meteo.columns}\n")

for col in df_marine.columns:
    print(f"{col}:  {type(col)}")


Colonnes df_marine:
Index(['id', 'wind_direction', 'wind_speed', 'wind_gust', 'pressure',
       'air_temperature', '3hr_pressure_tendency', 'Datetime', 'Station ID',
       'Lat', 'Lon'],
      dtype='object')

Colonnes df_meteo:
Index(['id', 'Datetime', 'temperature_2m', 'relative_humidity_2m',
       'dew_point_2m', 'precipitation', 'rain', 'showers', 'pressure_msl',
       'surface_pressure', 'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid',
       'cloud_cover_high', 'visibility', 'wind_speed_10m',
       'soil_temperature_0cm', 'soil_moisture_0_to_1cm', 'is_day'],
      dtype='object')

id:  <class 'str'>
wind_direction:  <class 'str'>
wind_speed:  <class 'str'>
wind_gust:  <class 'str'>
pressure:  <class 'str'>
air_temperature:  <class 'str'>
3hr_pressure_tendency:  <class 'str'>
Datetime:  <class 'str'>
Station ID:  <class 'str'>
Lat:  <class 'str'>
Lon:  <class 'str'>


In [14]:
print(f"{df_final.shape[0]} rows, {df_final.shape[1]} columns\n")
for col in df_final.columns:
    print(f"Col '{col}' : {df_final[col].isna().sum()}")
print(f"\n{df_final.dtypes}")

40857 rows, 34 columns

Col 'id_x' : 0
Col 'wind_direction' : 0
Col 'wind_speed' : 0
Col 'wind_gust' : 1091
Col 'wave_height' : 13091
Col 'average_wave_period' : 13091
Col 'dominant_wave_direction' : 15275
Col 'pressure' : 0
Col 'air_temperature' : 5460
Col 'water_temperature' : 10900
Col 'dewpoint' : 9822
Col 'Datetime' : 0
Col 'Station ID' : 0
Col 'Lat' : 0
Col 'Lon' : 0
Col 'id_y' : 0
Col 'temperature_2m' : 0
Col 'relative_humidity_2m' : 0
Col 'dew_point_2m' : 0
Col 'precipitation' : 0
Col 'rain' : 0
Col 'showers' : 0
Col 'pressure_msl' : 0
Col 'surface_pressure' : 0
Col 'cloud_cover' : 0
Col 'cloud_cover_low' : 0
Col 'cloud_cover_mid' : 0
Col 'cloud_cover_high' : 0
Col 'visibility' : 0
Col 'wind_speed_10m' : 0
Col 'soil_temperature_0cm' : 0
Col 'soil_moisture_0_to_1cm' : 0
Col 'is_day' : 0
Col '3hr_pressure_tendency' : 36495

id_x                                           int64
wind_direction                               float64
wind_speed                                   float64

In [17]:
# import pandas as pd

# def handle_null_values(df: pd.DataFrame) -> pd.DataFrame:
#     row_count = df.shape[0]
    
#     # Initialisation des listes pour suivre les colonnes supprimées
#     removed_columns = []
#     non_numeric_columns_to_drop = []
    
#     # Utiliser lambda et apply() pour calculer le nombre de valeurs nulles dans chaque colonne
#     null_counts = df.apply(lambda col: int(col.isnull().sum()))  # Calculer le nombre de NaN par colonne
    
#     # Condition : 1. Colonnes avec toutes les valeurs nulles ou 2. Plus de 50% de valeurs nulles et colonne non numérique
#     columns_to_drop = null_counts[
#         (null_counts == row_count) | 
#         ((null_counts > row_count * 0.5) & ~df.apply(lambda col: pd.api.types.is_numeric_dtype(col)))
#     ].index
    
#     # Ajouter les noms des colonnes supprimées dans les listes appropriées
#     for col in columns_to_drop:
#         if null_counts[col] == row_count:
#             removed_columns.append(col)  # Colonnes entièrement vides
#         elif null_counts[col] > row_count * 0.5 and not pd.api.types.is_numeric_dtype(df[col]):
#             non_numeric_columns_to_drop.append(col)  # Colonnes > 50% nulles et non numériques
    
#     # Supprimer les colonnes identifiées
#     df = df.drop(columns=columns_to_drop)
    
#     # Afficher les résultats
#     print("Colonnes supprimées pour avoir toutes les valeurs nulles:")
#     print(removed_columns)
    
#     print("\nColonnes supprimées pour avoir plus de 50% de valeurs nulles et être non numériques:")
#     print(non_numeric_columns_to_drop)
    
#     return df

# # Exemple d'utilisation
# # df_final = pd.read_csv('ton_fichier.csv') # Assure-toi que df_final est bien un DataFrame valide avant d'appeler la fonction
# df_final = handle_null_values(df_final)


In [18]:
# df_final = df_final.round(2)
# print(df_final.columns)
# df_final.describe()

In [19]:
# def explore_dict_keys(d, parent_key='', sep='_'):
#     """
#     Explore un dictionnaire récursivement pour obtenir toutes les clés, y compris les sous-clés,
#     mais ne retourne pas les valeurs finales.

#     :param d: Le dictionnaire à explorer
#     :param parent_key: La clé parent qui est utilisée pour concaténer les sous-clés
#     :param sep: Le séparateur utilisé pour concaténer les clés (par défaut '_')
#     :return: Une liste des clés (et sous-clés)
#     """
#     keys = []
#     for k, v in d.items():
#         new_key = f"{parent_key}{sep}{k}" if parent_key else k
#         if isinstance(v, dict):  # Si la valeur est un dictionnaire, on explore récursivement
#             keys.append(new_key)  # Ajouter la clé, mais ne pas inclure la valeur
#             keys.extend(explore_dict_keys(v, new_key, sep=sep))  # Continuer l'exploration
#         else:
#             keys.append(new_key)  # Ajouter la clé finale
#     return keys

In [20]:
# def find_key_path(d, target_key, path=[]):
#     """
#     Recherche récursive d'une clé dans un dictionnaire et retourne son chemin.
#     :param d: dictionnaire
#     :param target_key: clé recherchée
#     :param path: liste pour stocker le chemin jusqu'à la clé
#     :return: chemin sous forme de liste
#     """
#     if isinstance(d, dict):  # Si le dictionnaire est encore imbriqué
#         for key, value in d.items():
#             new_path = path + [key]
#             if key == target_key:
#                 return new_path
#             elif isinstance(value, dict):
#                 result = find_key_path(value, target_key, new_path)
#                 if result:  # Si la clé est trouvée, retourner le chemin
#                     return result
#     return None  # Retourne None si la clé n'a pas été trouvée



# # Recherche du chemin pour la clé 'marine_data'
# path = find_key_path(table_dict, "Marine Dataframe")
# print(path)


Auto_convert Test

In [21]:
# for idx, (buoy_id, tables) in enumerate(table_dict.items()):  # Utilisation de .items() pour obtenir (clé, valeur)
#     if isinstance(tables, dict):
#         if idx == 1:  # Vérifier si l'index est égal à 1

Counting Rows of all Dataframes in total